In [55]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from shapely.geometry import Point
import pandas as pd
from dotenv import dotenv_values
import logging
import folium

from generator_drainage_units.generator_order_levels import GeneratorOrderLevels

logging.basicConfig(level=logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
# Define case and base_dir

config = dotenv_values("..\\.env")
base_dir = config["BASE_DIR"]
# case_name = "vallei_en_veluwe"
# case_name = "geerestein"
# case_name = "hattemerbroek"
# case_name = "pangelerbeek"
case_name = "Leuvenumse_beek"

In [57]:
# Load class and accompanying data as object 'case_data' for selected case

def run_generator_order_levels(
    path: Path,
    read_results: bool = True,
    write_results: bool = True
) -> GeneratorOrderLevels:
    
    order_generator = GeneratorOrderLevels(
        read_results=read_results, write_results=write_results
    )
    order_generator.read_data_from_case(path=path)
    
    return order_generator

case_path = Path(base_dir, case_name)

case_data = run_generator_order_levels(path=case_path)

INFO:root: ### Case "Leuvenumse_beek" ###
INFO:root:   x read basisdata


Step 1: Find dead-end end nodes

In [80]:
# Copy hydroobject data to new variable 'hydroobjects' and make dataframes with start and end nodes
hydroobjects = case_data.hydroobjecten[["CODE", "geometry"]].copy()
hydroobjects["start_node"] = hydroobjects.geometry.apply(lambda x: Point(x.coords[0][0:2]))
hydroobjects["end_node"] = hydroobjects.geometry.apply(lambda x: Point(x.coords[-1][0:2]))

start_nodes = hydroobjects[["start_node"]].rename(columns={"start_node": "geometry"})
end_nodes = hydroobjects[["end_node"]].rename(columns={"end_node": "geometry"})

In [81]:
# Determine dead-end end nodes (using buffer to deal with inaccuracies in geometry hydroobjects) and make new dataframe
buffer_width = 0.5
buffer_rws = 10.0

start_nodes['geometry'] = start_nodes.buffer(buffer_width)
end_nodes['geometry'] = end_nodes.buffer(buffer_width)

# Determine where start and end nodes overlap
dead_end_nodes = start_nodes.sjoin(end_nodes, how="right")
dead_end_nodes = hydroobjects.loc[dead_end_nodes[dead_end_nodes.index_left.isna()].index, ["CODE", "end_node"]]
dead_end_nodes = dead_end_nodes.rename(columns={"end_node": "geometry"})

dead_end_nodes["geometry"] = dead_end_nodes["geometry"].buffer(buffer_width)
sjoin = dead_end_nodes.sjoin(hydroobjects[["CODE", "geometry"]])
no_dead_end_node_ids = sjoin[sjoin["CODE_left"]!=sjoin["CODE_right"]].index

dead_end_nodes = dead_end_nodes[~dead_end_nodes.index.isin(no_dead_end_node_ids)]
dead_end_nodes.geometry = dead_end_nodes.geometry.centroid

rws_wateren = case_data.rws_wateren.copy()
rws_wateren.geometry = rws_wateren.geometry.buffer(buffer_rws)
outflow_nodes = dead_end_nodes.sjoin(rws_wateren[["geometry", "rws_code"]]).drop(columns="index_right").reset_index(drop=True)

In [82]:
outflow_nodes = outflow_nodes.rename(columns={"CODE": "hydroobject_code"})
outflow_nodes_all = None
for rws_code in outflow_nodes.rws_code.unique():
    outflows = outflow_nodes[outflow_nodes.rws_code==rws_code].reset_index(drop=True)
    outflows["rws_code_no"] = outflows.index
    outflows["orde_code"] = outflows.apply(lambda x: f"{x.rws_code}.{str(x.rws_code_no + 1).zfill(3)}", axis=1)
    if outflow_nodes_all is None:
        outflow_nodes_all = outflows
    else:
        outflow_nodes_all = pd.concat([outflow_nodes_all, outflows])
outflow_nodes_all

,hydroobject_code,geometry,rws_code,no,orde_code
0,WL_1990,POINT (175924.819 488149.53),VE,0,VE.001
1,WL_2014,POINT (175564.894 487919.658),VE,1,VE.002
2,WL_279,POINT (173313.954 486529.476),VE,2,VE.003
3,WL_1655,POINT (173945.582 487036.095),VE,3,VE.004
4,WL_1773,POINT (177294.183 488900.604),VE,4,VE.005
5,WL_1777,POINT (176878.465 488799.623),VE,5,VE.006
6,WL_105016,POINT (173120.242 486356.852),VE,6,VE.007
7,WL_1702,POINT (174725.355 487916.543),VE,7,VE.008


In [45]:
# Make figure
outflow_nodes_4326 = outflow_nodes.to_crs(4326)

map = folium.Map(
    location=[
        outflow_nodes_4326.geometry.y.mean(), 
        outflow_nodes_4326.geometry.x.mean()
    ], 
    zoom_start=13
)

folium.GeoJson(
    case_data.hydroobjecten.geometry,#.buffer(10),
    name="Watergangen",
    color="blue",
    fill_color="blue",
    zoom_on_click=True,
).add_to(map)

folium.GeoJson(
    dead_end_nodes,
    name="Outflow points",
    marker=folium.Circle(radius=25, fill_color="orange", fill_opacity=0.4, color="orange", weight=3),
    highlight_function=lambda x: {"fillOpacity": 0.8},
    zoom_on_click=True,
).add_to(map)

folium.GeoJson(
    outflow_nodes_4326,
    name="Overlap",
    marker=folium.Circle(radius=1, fill_color="red", fill_opacity=0.4, color="red", weight=3),
    highlight_function=lambda x: {"fillOpacity": 0.8},
    zoom_on_click=True,
).add_to(map)

folium.GeoJson(
    case_data.rws_wateren.geometry,
    name = "RWS_Wateren",
    
).add_to(map)

map

Step 2: Link dead-end end nodes to RWS waterbodies

Step 3: Assign code to dead-end end nodes 